# 예제로 본 간단한 자동미분

In [1]:
from typing import Union, List

import numpy as np

np.set_printoptions(precision=4)

In [2]:
a = 3
a.__add__(4)

7

In [3]:
a = np.array([2,3,1,0])

print(a)
print("Addition using '__add__':", a.__add__(4))
print("Addition using '+':", a + 4)

[2 3 1 0]
Addition using '__add__': [6 7 5 4]
Addition using '+': [6 7 5 4]


In [3]:
Numberable = Union[float, int]

def ensure_number(num: Numberable):
    if isinstance(num, NumberWithGrad):
        return num
    else:
        return NumberWithGrad(num)        

class NumberWithGrad(object):
    
    def __init__(self, 
                 num: Numberable,
                 depends_on: List[Numberable] = None,
                 creation_op: str = ''):
        self.num = num
        self.grad = None
        self.depends_on = depends_on or []
        self.creation_op = creation_op

    def __add__(self, 
                other: Numberable):
        return NumberWithGrad(self.num + ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'add')
    
    def __mul__(self,
                other: Numberable = None):

        return NumberWithGrad(self.num * ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'mul')
    
    def backward(self, backward_grad: Numberable = None):
        if backward_grad is None: # backward가 처음 호출됨
            self.grad = 1
        else: 
            # 이 부분에서 기울기가 누적됨
            # 기울기 정보가 아직 없다면 backward_grad로 설정
            if self.grad is None:
                self.grad = backward_grad
            # 기울기 정보가 있다면 기존 기울깃값에 backward_grad를 더함
            else:
                self.grad += backward_grad
        
        if self.creation_op == "add":
            # self.grad를 역방향으로 전달함 
            # 둘 중 어느 요소를 증가시켜도 출력이 같은 값만큼 증가함
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad)    

        if self.creation_op == "mul":

            # 첫 번째 요소에 대한 미분 계산
            new = self.depends_on[1] * self.grad
            # 이 요소에 대한 미분을 역방향으로 전달
            self.depends_on[0].backward(new.num)

            # 두 번째 요소에 대한 미분 계산
            new = self.depends_on[0] * self.grad
            # 이 요소에 대한 미분을 역방향으로 전달
            self.depends_on[1].backward(new.num)

In [4]:
a = NumberWithGrad(3)
b = a * 4
c = b + 3
c.backward()
print(a.grad) # 예상한 값이 출력됨
print(b.grad) # 예상한 값이 출력됨

4
1


In [5]:
a = NumberWithGrad(3)

In [6]:
b = a * 4
c = b + 3
d = (a + 2)
e = c * d 
e.backward() 

In [7]:
a.grad # 예상한 값이 출력됨

35